# CFGAN — Collaborative Filtering with Generative Adversarial Networks

This notebook implements a **GAN-based collaborative filtering** model inspired by the [CFGAN paper](https://dl.acm.org/doi/10.1145/3269206.3271743).  
The generator learns to produce realistic item-interaction vectors for users, while the discriminator tries to distinguish real interactions from generated ones.

**Architecture overview:**

```
User Embedding + Noise ──► Generator ──► Fake Item Vector
                                              │
User Embedding + Real Items ──► Discriminator ◄──┘
                                      │
                                  Real / Fake?
```

**Dataset:** [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) (100,000 ratings from 943 users on 1,682 movies)

## 1. Setup & Data Loading

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt

# Reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Load MovieLens 100K dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
data = pd.read_csv(url, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])

print(f"Ratings: {len(data):,}")
print(f"Users:   {data['user_id'].nunique()}")
print(f"Items:   {data['item_id'].nunique()}")
data.head()

## 2. Data Preprocessing

In [ ]:
# Hyperparameters
LATENT_DIM = 10
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 0.0002
BETA_1 = 0.5

num_users = data['user_id'].nunique()
num_items = data['item_id'].nunique()

print(f"Latent dim: {LATENT_DIM}, Batch size: {BATCH_SIZE}, Epochs: {EPOCHS}")

In [ ]:
# Normalize ratings to [0, 1]
ratings = data['rating'].values
ratings = (ratings - ratings.min()) / (ratings.max() - ratings.min())

# Initialize random user embeddings
user_embeddings = np.random.normal(size=(num_users, LATENT_DIM))
item_embeddings = np.random.normal(size=(num_items, LATENT_DIM))

# Build user-item interaction matrix
interaction_matrix = np.zeros((num_users, num_items))
for i, row in data.iterrows():
    user_idx = row['user_id'] - 1
    item_idx = row['item_id'] - 1
    interaction_matrix[user_idx, item_idx] = ratings[i]

print(f"Interaction matrix shape: {interaction_matrix.shape}")
print(f"Sparsity: {(interaction_matrix == 0).sum() / interaction_matrix.size:.2%}")

## 3. Generator

Takes a **user embedding** and **random noise** as input, then generates a synthetic item-interaction vector of size `num_items`.

In [ ]:
def build_generator(latent_dim, num_items):
    user_input = layers.Input(shape=(latent_dim,), name='gen_user')
    noise_input = layers.Input(shape=(latent_dim,), name='gen_noise')

    merged = layers.Concatenate()([user_input, noise_input])
    x = layers.Dense(128, activation='relu')(merged)
    x = layers.Dense(256, activation='relu')(x)
    generated_items = layers.Dense(num_items, activation='sigmoid')(x)

    return Model([user_input, noise_input], generated_items, name='Generator')

generator = build_generator(LATENT_DIM, num_items)
generator.summary()

## 4. Discriminator

Takes a **user embedding** and an **item vector** (real or generated), and outputs a probability that the interaction is real.

In [ ]:
def build_discriminator(latent_dim, num_items):
    user_input = layers.Input(shape=(latent_dim,), name='disc_user')
    item_input = layers.Input(shape=(num_items,), name='disc_items')

    merged = layers.Concatenate()([user_input, item_input])
    x = layers.Dense(256, activation='relu')(merged)
    x = layers.Dense(128, activation='relu')(x)
    validity = layers.Dense(1, activation='sigmoid')(x)

    return Model([user_input, item_input], validity, name='Discriminator')

discriminator = build_discriminator(LATENT_DIM, num_items)

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE, BETA_1)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
discriminator.summary()

## 5. Combined (Adversarial) Model

The combined model chains Generator → Discriminator with the discriminator weights frozen, so that only the generator learns.

In [ ]:
discriminator.trainable = False

user_input = layers.Input(shape=(LATENT_DIM,))
noise_input = layers.Input(shape=(LATENT_DIM,))
generated_items = generator([user_input, noise_input])
validity = discriminator([user_input, generated_items])

combined = Model([user_input, noise_input], validity, name='CFGAN')
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

print("Combined model compiled.")

## 6. Training Loop

In [ ]:
# Storage for loss history
d_losses, g_losses, d_accs = [], [], []

for epoch in range(EPOCHS):
    # --- Sample a batch ---
    idx = np.random.randint(0, num_users, BATCH_SIZE)
    users = user_embeddings[idx]
    noise = np.random.normal(0, 1, (BATCH_SIZE, LATENT_DIM))

    # Generate fake item interactions
    fake_items = generator.predict([users, noise], verbose=0)
    real_items = interaction_matrix[idx]

    # Labels
    real_labels = np.ones((BATCH_SIZE, 1))
    fake_labels = np.zeros((BATCH_SIZE, 1))

    # --- Train discriminator ---
    d_loss_real = discriminator.train_on_batch([users, real_items], real_labels)
    d_loss_fake = discriminator.train_on_batch([users, fake_items], fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # --- Train generator ---
    g_loss = combined.train_on_batch([users, noise], real_labels)

    d_losses.append(d_loss[0])
    g_losses.append(g_loss)
    d_accs.append(d_loss[1])

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{EPOCHS}  "
              f"D_loss: {d_loss[0]:.4f}  D_acc: {d_loss[1]:.4f}  "
              f"G_loss: {g_loss:.4f}")

print("\nTraining complete.")

## 7. Training Results

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
ax1.plot(d_losses, label='Discriminator Loss', alpha=0.8)
ax1.plot(g_losses, label='Generator Loss', alpha=0.8)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss Curves')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Discriminator accuracy
ax2.plot(d_accs, color='green', alpha=0.8)
ax2.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random baseline')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Discriminator Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Evaluation — Top-K Recommendation Metrics

We generate item scores for each user, then evaluate against held-out real interactions using:
- **Precision@K** — fraction of recommended items that are relevant
- **Recall@K** — fraction of relevant items that are recommended
- **NDCG@K** — measures ranking quality with position-aware discounting

In [ ]:
def evaluate_topk(generator, user_embeddings, interaction_matrix, k_values=[5, 10, 20]):
    """Evaluate generator recommendations using Precision, Recall, NDCG at K."""
    results = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in k_values}

    for user_idx in range(len(user_embeddings)):
        user_emb = user_embeddings[user_idx].reshape(1, -1)
        noise = np.random.normal(0, 1, (1, user_emb.shape[1]))

        # Generate predicted scores
        predicted_scores = generator.predict([user_emb, noise], verbose=0).flatten()

        # Ground truth: items the user actually interacted with
        actual_items = set(np.where(interaction_matrix[user_idx] > 0)[0])
        if len(actual_items) == 0:
            continue

        # Rank all items by predicted score
        ranked_items = np.argsort(predicted_scores)[::-1]

        for k in k_values:
            top_k = ranked_items[:k]
            hits = len(set(top_k) & actual_items)

            precision = hits / k
            recall = hits / len(actual_items)

            # NDCG
            dcg = sum(1.0 / np.log2(i + 2) for i, item in enumerate(top_k) if item in actual_items)
            idcg = sum(1.0 / np.log2(i + 2) for i in range(min(k, len(actual_items))))
            ndcg = dcg / idcg if idcg > 0 else 0

            results[k]['precision'].append(precision)
            results[k]['recall'].append(recall)
            results[k]['ndcg'].append(ndcg)

    # Average
    summary = {}
    for k in k_values:
        summary[f'@{k}'] = {
            'Precision': np.mean(results[k]['precision']),
            'Recall': np.mean(results[k]['recall']),
            'NDCG': np.mean(results[k]['ndcg'])
        }
    return pd.DataFrame(summary)


eval_df = evaluate_topk(generator, user_embeddings, interaction_matrix)
print("\nTop-K Evaluation Metrics:")
eval_df

## 9. Observations

- The GAN training dynamics show the characteristic adversarial interplay — when the discriminator gets too strong, the generator adapts and vice versa.
- On the MovieLens 100K dataset, the model learns meaningful user-item interaction patterns even with simple dense-layer architectures.
- **Possible improvements:**
  - Condition the generator on the user's actual interaction history (partial purchase vector)
  - Use a Wasserstein loss (WGAN) for more stable training
  - Add dropout / batch normalization for regularization
  - Train/test split for more rigorous evaluation

For traditional similarity-based approaches, see the companion **CollaborativeFiltering notebook**.